# Load and Preprocess Data 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D , Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import numpy as np
from collections import Counter
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    "D:/Senior WorkTable/GrowQuest_Data_Final/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    "D:/Senior WorkTable/GrowQuest_Data_Final/val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    "D:/Senior WorkTable/GrowQuest_Data_Final/test",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


# Load and Fine-Tune EfficientNet

In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Freeze base model initially
base_model.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Fine-Tune

Unfreeze base layers and fine-tune at lower LR:

In [ ]:
all_classes = np.concatenate([
    train_gen.classes,
    val_gen.classes,
    test_gen.classes
])

# Count how many images per class
class_counts = Counter(all_classes)

# Get class label names
class_labels = train_gen.class_indices
idx_to_class = {v: k for k, v in class_labels.items()}

# Print results
for class_idx, count in class_counts.items():
    print(f"Class '{idx_to_class[class_idx]}': {count} images")

In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)  # Add dropout layer
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)                 
x = Dropout(0.5)(x)  # Add dropout layer
output = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Freeze base model initially
base_model.trainable = False


In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',     # Monitor validation loss
    patience=3,             # Stop after 3 epochs with no improvement
    restore_best_weights=True # Restore weights from the best epoch
)

reduce_lr = ReduceLROnPlateau (monitor='val_loss',
                            factor=0.5,
                            patience=2, 
                            min_lr=1e-6)


In [ ]:
base_model.trainable = True
model.compile(optimizer=Adam(learning_rate=1e-3), 
            loss='categorical_crossentropy', 
            metrics=['accuracy'],
)

history = model.fit(train_gen, 
                    validation_data=val_gen,
                    epochs=10,
                    callbacks=[early_stop,reduce_lr])


In [ ]:
import matplotlib.pyplot as plt

# Plot Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy over Epochs')
plt.grid(True)
plt.show()

# Plot Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss over Epochs')
plt.grid(True)
plt.show()
